Lnagchain composes chains of components

ICEL and the runnable protocol define:

1. An allowed set of input types 

2. An allowed set of output types

3. Standard methods (invoke, stream, batch) - Means of modifying parameters at runtime (bind)

syntax looks like

chain = prompt | llm | OutputParser.

Interface :- 

1. Components implement Runnable protocol

2. Common methods includes:- 
    
    1. Invoke -synchronus method [ainvoke] -asynchronus
    
    2. stream -synchronus method [astream] -asynchronus

    3. batch -synchronus method [abatch] -asynchronus

3. Common properties:- 

    input_schema, output_schema

4. Common I/O

Component           |Input Type                                 |Output Type|
|:--:|:--:     |:--:|
Prompt              |Dictionary                                 |Prompt Value
|:--:|:--:     |:--:|
Retriever           |Single String                              |List of Documents
|:--:|:--:     |:--:|
LLM                 |String, List of messages or prompt value   |String
|:--:|:--:     |:--:|
ChatModel           |String, List of messages or prompt value   |ChatMessage
|:--:|:--:     |:--:|
Tool                |String/Dictionary                          |Tool Dependent
|:--:|:--:     |:--:|
OutputParser        |Output or LLM or ChatModel                 |Parser dependent
|:--:|:--:     |:--:|


Why use LCEL :- 

Runnables Support:-

1. Async, Batch and Streaming support

2. Fallbacks

3. Parallelism
    1. LLM calls can be time consuming
    2. Any components that can be run in parallel are!

4. Logging is built in